In [30]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
import json

In [ ]:
try:
    wd = os.path.dirname(os.path.abspath(__file__))
except:
    wd = os.getcwd()

wd = wd.split("/")
repo_path_idx = wd.index("fineprint")
repo_path = "/".join(wd[:repo_path_idx + 1])

/Users/emmajuansalazar/arxius/projects/fineprint


# 1. Summarizer dataset

Same as it came originally

# 2. ToS dataset

In [16]:
categories = False
try:
    cuad_categories = pd.read_csv(f"{repo_path}/curating-datasets/CUAD_categories.csv")
    categories = True
except:
    print("Categories file not computed. Computing now...")

In [15]:
if not categories:
    with open(f"{repo_path}/data/candidate-datasets/CUAD_v1/CUAD_v1_README.txt") as f:
        lines = f.readlines()

    category_list_i = lines.index("CATEGORY LIST\n")
    next_section_i = lines.index("SOURCE OF CONTRACTS\n")
    cuad_categories = lines[category_list_i:next_section_i]
    cuad_categories = [x.replace("\t", "").replace("\n", "").strip() for x in cuad_categories]
    cuad_categories = [x for x in cuad_categories if len(x) > 0]

    cuad_categories_dict = {'category': [], 'description': [], 'answer_format': []}

    i = 0
    while i < len(cuad_categories):
        if cuad_categories[i].isdigit():
            category = cuad_categories[i+1].split(":")[1].strip()
            description = cuad_categories[i+2].split(":")[1].strip()
            answer_format = cuad_categories[i+3].split(":")[1].strip()

            cuad_categories_dict['category'].append(category)
            cuad_categories_dict['description'].append(description)
            cuad_categories_dict['answer_format'].append(answer_format)
            i += 4
        i += 1

    cuad_categories = pd.DataFrame(cuad_categories_dict)
    cuad_categories.to_csv(f"{repo_path}/curating-datasets/CUAD_categories.csv", index=False)

In [18]:
cuad_categories

,category,description,answer_format
0,Document Name,The name of the contract,Contract Name
1,Parties,The two or more parties who signed the contract,Entity or individual names
2,Agreement Date,The date of the contract,Date (mm/dd/yyyy)
3,Effective Date,The date when the contract is effective,Date (mm/dd/yyyy)
4,Expiration Date,On what date will the contract's initial term ...,Date (mm/dd/yyyy) / Perpetual
5,Renewal Term,What is the renewal term after the initial ter...,[Successive] number of years/months / Perpetual
6,Notice to Terminate Renewal,What is the notice period required to terminat...,Number of days/months/year(s)
7,Governing Law,Which state/country's law governs the interpre...,"Name of a US State / non-US Province, Country"
8,Most Favored Nation,Is there a clause that if a third party gets b...,Yes/No
9,Non-Compete,Is there a restriction on the ability of a par...,Yes/No


We're only interested in the Yes/No categories, since they indicate wether the clause belongs to that category.

In [5]:
cuad = pd.read_csv(f"{repo_path}/data/candidate-datasets/CUAD_v1/master_clauses.csv")

In [19]:
cuad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 83 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Filename                                    510 non-null    object
 1   Document Name                               510 non-null    object
 2   Document Name-Answer                        510 non-null    object
 3   Parties                                     510 non-null    object
 4   Parties-Answer                              509 non-null    object
 5   Agreement Date                              510 non-null    object
 6   Agreement Date-Answer                       465 non-null    object
 7   Effective Date                              510 non-null    object
 8   Effective Date-Answer                       359 non-null    object
 9   Expiration Date                             510 non-null    object
 10  Expiration Date-Answer    

In [20]:
cuad_filtered = cuad.iloc[:, [0] + list(range(17,75)) + list(range(77, 83))]

In [21]:
cuad_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 65 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   Filename                                  510 non-null    object
 1   Most Favored Nation                       510 non-null    object
 2   Most Favored Nation-Answer                510 non-null    object
 3   Competitive Restriction Exception         510 non-null    object
 4   Competitive Restriction Exception-Answer  510 non-null    object
 5   Non-Compete                               510 non-null    object
 6   Non-Compete-Answer                        510 non-null    object
 7   Exclusivity                               510 non-null    object
 8   Exclusivity-Answer                        510 non-null    object
 9   No-Solicit Of Customers                   510 non-null    object
 10  No-Solicit Of Customers-Answer            510 non-

In [22]:
cuad_columns = list(cuad.columns)

In [26]:
tos_dataset = []

rows, columns = cuad.shape
for i in tqdm(range(rows)):
    row = cuad.iloc[i]
    clause_cols = [num for num in range(2, columns, 2)]
    for j in clause_cols:
        if cuad.iloc[i, j] == "Yes":
            clause_dict = {'clause': cuad.iloc[i, j-1],
                           'label': cuad_columns[j-1]}
            tos_dataset.append(clause_dict)

100%|██████████| 510/510 [00:00<00:00, 2212.55it/s]


In [29]:
tos_dataset

[{'clause': "['Company shall not specify the business practices of MA, nor regulate the manner in which MA shall operate its business, provided that MA (a) conducts business in a manner that reflects favorably at all times on the Technology sold and the good name, goodwill and reputation of Company and its affiliates<omitted>']",
  'label': 'Non-Disparagement'},
 {'clause': "['MA may not assign, sell, lease or otherwise transfer in whole or in party any of the rights granted pursuant to this Agreement without prior written approval of Company.']",
  'label': 'Anti-Assignment'},
 {'clause': "['INITIAL ORDER COMMITMENT - MA commits to purchase a minimum of 100 Units in aggregate within the Territory within the first six months of term of this Agreement.']",
  'label': 'Minimum Commitment'},
 {'clause': '[\'Company hereby grants MA, during the term of this Agreement, the right to use Company and/or Company trade names, trademarks or service marks on Technology or in advertising or promoti

In [31]:
output_file = f"{repo_path}/data/stage2-analyzing/tos_dataset.json"

with open(output_file, 'w') as json_file:
    json.dump(tos_dataset, json_file, indent=4)

# 3. PP dataset